<a href="https://colab.research.google.com/github/Desi-Noviana/Analisis-Sentimen-Ojek-Online/blob/main/ScrapingOnline_Review_Ojol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
pip install google_play_scraper

     |████████████████████████████████| 52 kB 1.0 MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.3-py3-none-any.whl size=24394 sha256=b31db3227997ac610a93afc505be328a2cdf6fc2bf073e1ac76dbd16462d86bd
  Stored in directory: /root/.cache/pip/wheels/81/37/0b/4a14be55b449a048cd93d79930b1a980dee7896480defa1923
Successfully built google-play-scraper


In [ ]:
import json
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter
from google_play_scraper import Sort, reviews, app
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [ ]:
app_packages = [
  'com.gojek.app'
]

In [ ]:
app_infos = []
for ap in tqdm(app_packages):
  info = app(ap, lang='id', country='id')
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  3.93it/s]


In [ ]:
def print_json(json_object):
  json_str = json.dumps(
    json_object,
    indent=2,
    sort_keys=True,
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [ ]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 dan yang lebih tinggi",
  "appId": "com.gojek.app",
  "containsAds": false,
  "contentRating": "Rating 3+",
  "contentRatingDescription": null,
  "currency": "IDR",
  "description": "Gojek bukan hanya aplikasi penyedia layanan transportasi, pesan antar makanan, logistik, pembayaran, dan kebutuhan sehari-hari, loh.\r\n\r\nGojek juga punya misi sosial untuk meningkatkan taraf hidup masyarakat. Caranya? Dengan saling membantu!\r\n\r\nSampai hari ini, Gojek telah bermitra dengan lebih dari 1 juta driver, 125 ribu pedagang kuliner, dan 30 ribu penyedia jasa lainnya, semua tersebar di 50 kota di Indonesia.\r\n\r\nNah, dengan men-download aplikasi dan menggunakan jasa Gojek, kamu gak cuma bakal terbantu dalam mengerjakan urusan kamu sehari-hari\u2026 tapi kamu juga bakal membantu para mitra Gojek dalam menggapai mimpi-mimpi mereka dan keluarga mereka, dan turut jadi bagian dari misi meningkatkan taraf hidup masya

In [ ]:
app_reviews = []
for ap in tqdm(app_packages):
  for score in list(range(5, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='id',
        country='id',
        sort=sort_order,
        count= 9000 if score == 5 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:26<00:00, 26.09s/it]


In [ ]:
print_json(app_reviews[0])

{
  "appId": "com.gojek.app",
  "at": "2022-03-18 06:03:16",
  "content": "Sebelum di update sudah ada \"BUG\"nya, setelah di update malah tambah banyak!!! Mau logout malah tidak bisa, layar tertutup tulisan-tulisan TIDAK JELAS, tidak bisa di geser atau dipindah!!! Mana secara tampilan tidak ada perubahan, fungsi-fungsinya masih sama. Lama-lama aku UNINSTAL aplikasi ini!!!",
  "repliedAt": "2021-12-06 08:21:59",
  "replyContent": "Hai Setiyo Armanu, mohon maaf untuk kendalanya. Laporan yang kamu sampaikan akan menjadi masukan bagi kami untuk terus meningkatkan kualitas layanan kami ke depannya. Apabila kamu mengalami kendala terkait layanan Gojek, silakan laporkan melalui menu Bantuan di aplikasi Gojek agar tim kami bisa bantu cek ya. ^Ardi",
  "reviewCreatedVersion": "4.39.2",
  "reviewId": "gp:AOqpTOGKp4kpTIB4kXX9xKbbgi_Dcg4mUr-XGO-nreiU9FgOFy2dqoEvm6bmO64Ep0xBlGLZvDmKgLVSZfEg2w",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 1224,
  "userImage": "https://play-lh.g

In [ ]:
len(app_reviews)

18000

In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviews.csv', index=None, header=True)